In [1]:
import pandas as pd
import re
import json
import os

In [2]:
f = open('train_2.txt', 'r', encoding = 'utf-8')
words = f.read()
word_list = words.split('--------------------\n\n')

my_map = {'name':'名字', 'location':'地點', 'time':'時間', 'contact':'聯絡方式', 'id':'編號', 
    'profession':'職業', 'biomarker':'個人生物標誌', 'family':'家庭成員', 'clinical_event':'有名的臨床事件',
    'special_skills':'特殊專業或技能', 'unique_treatment':'獨家或聞名的治療方法', 'account':'帳號', 
    'organization':'所屬團體', 'education':'就學經歷或學歷', 'money':'金額', 'belonging_mark':'所屬品的特殊標誌',
    'med_exam':'報告數值', 'others':'其他'}
inv_map = {v: k for k, v in my_map.items()}



def split_conser_label(input_list):
    # input word_list
    convers = []
    label = []
    for i in input_list:
        j = i.split('\n', 1)
        if len(j[0]) > 0:
            convers.append(j[0])
        if len(j[1].split('\n\n')[0]):
            label.append(j[1].split('\n\n')[0])
    return convers, label


def label_to_dict(input_label_list):
    # input label
    dict_label = {}
    for i in range(len(input_label_list)):
        total_label = input_label_list[i].split('\n')
        dict_label.update({i : dict()})
        for j in range(len(total_label)-1):
            each_row = total_label[j+1].split('\t')
            duration = (int(each_row[2]) - int(each_row[1]))
            for q in range(duration):
                # origin_label = each_row[4].lower()
                # chinese_label = my_map[origin_label]
                dict_label[i].update({int(each_row[1])+q : each_row[4]})
    return dict_label




def fit_ner_form(input_con, input_label_dict):
    #input convers dict_label
    list_sentence_id = []
    list_words = []
    list_labels = []

    for i in range(len(input_con)):
        now = input_con[i]
        for j in range(len(now)):
            if j in input_label_dict[i].keys():
                list_sentence_id.append(i)
                list_words.append(now[j])
                list_labels.append(input_label_dict[i][j])
            else:
                list_sentence_id.append(i)
                list_words.append(now[j])
                list_labels.append('O')
    return list_sentence_id, list_words, list_labels



convers, label = split_conser_label(word_list)
print('done convers, label')
dict_label = label_to_dict(label)
print('done dict_label')
list_sentence_id, list_words, list_labels = fit_ner_form(convers, dict_label)
print('done list_sentence_id')

train_df = pd.DataFrame({'sentence_id':list_sentence_id, 'words':list_words, 'labels':list_labels})


done convers, label
done dict_label
done list_sentence_id


In [3]:
train_df.head()

,sentence_id,words,labels
0,0,醫,O
1,0,師,O
2,0,：,O
3,0,啊,O
4,0,回,O


In [4]:
sent = []
labels = []
for i in list(set(train_df['sentence_id'].values)):
    df = train_df[train_df['sentence_id']==i]
    sent.append(''.join(df.words.values))
    labels.append(list(df.labels.values))

In [5]:
df_now = pd.DataFrame({'sentence':sent, 'label':labels})

In [6]:
def transfer_(row):
    list_label = []
    for i in range(len(row.sentence)):
        list_label.append({row.sentence[i]:row.label[i]})
    return list_label
df_now['label'] = df_now.apply(transfer_, 1)

In [7]:
re.findall('今天', df_now.iloc[0]['sentence'])

['今天', '今天', '今天']

In [8]:
df_now = df_now.reset_index()

In [9]:
df_now.columns = ['sentence_id', 'words', 'predict']

In [10]:
df = df_now.copy()

num = 0
sentence_id = []
words = []
predictions = []

for i in range(len(df)):
    if i==0:
        sentence_id.append(df.iloc[i]['sentence_id'])
        words.append(df.iloc[i]['words'])
        predictions.append(df.iloc[i]['predict'])
    else:
        if df.iloc[i]['sentence_id'] == sentence_id[num]:
            words[num]+= ' '+ df.iloc[i]['words']
                #print(words[num])
                #print(predictions[num])
            predictions[num].extend(df.iloc[i]['predict'])
        else:
            sentence_id.append(df.iloc[i]['sentence_id'])
            words.append(df.iloc[i]['words'])
            predictions.append(df.iloc[i]['predict'])
            num+=1
df = pd.DataFrame({'sentence_id':sentence_id, 'words':words, 'predictions':predictions})
print(len(df))

art_id = []
order_ = []
word_ = []
label_ = []

for i in range(len(df)):
    now = df.iloc[i]['predictions']
    for j in range(len(now)):
        if 'O' not in now[j].values():
            art_id.append(i)
            order_.append(j)
            for k, v in now[j].items():
                word_.append(k)
                label_.append(v)
                #label_.append(v.split('-')[1])

print(len(pd.DataFrame({'art_id':art_id, 'order_':order_, 'word_':word_, 'label_':label_})))


total = len(art_id)
i = 0
j = 0

article_id = []
start_position = []
end_position = []
entity_text = []
entity_type = []

# 'art_id':art_id, 'order_':order_, 'word_':word_, 'label_':label_
while i<total:
    if i == 0:
        article_id.append(art_id[i])
        start_position.append(int(order_[i]))
        end_position.append(int(order_[i])+1)
        entity_text.append(word_[i])
        entity_type.append(label_[i])
        i+=1
    else:
        if article_id[j] == art_id[i] and end_position[j] == int(order_[i]) and entity_type[j] == label_[i]:
            end_position[j] = int(order_[i])+1
            entity_text[j] += word_[i]
            i+=1
        else:
            article_id.append(art_id[i])
            start_position.append(int(order_[i]))
            end_position.append(int(order_[i])+1)
            entity_text.append(word_[i])
            entity_type.append(label_[i])
            i+=1
            j+=1
df = pd.DataFrame({'article_id':article_id, 'start_position':start_position, 'end_position':end_position,
                  'entity_text':entity_text, 'entity_type':entity_type})

200
11872


In [12]:
df[df['entity_text'].isin(['星期三'])]

,article_id,start_position,end_position,entity_text,entity_type
157,9,1363,1366,星期三,time
158,9,1367,1370,星期三,time


In [16]:

for i in range(len(df)):
    if '星期三' in df.entity_text.iloc[i]:
        print(df.entity_text.iloc[i])

星期三
星期三
每星期三四


In [14]:
count_total_num = 0
for i in range(len(df_now)):
    count_total_num += len(re.findall('今天', df_now.iloc[i]['words']))
count_total_num

450

In [35]:
def find_out_aidea(word):
    count_has_label = 0
    count_total_num = 0
    for i in range(len(df)):
        if word in df.entity_text.iloc[i]:
            count_has_label += 1
    #count_has_label
    
    for i in range(len(df_now)):
        count_total_num += len(re.findall(word, df_now.iloc[i]['words']))
    return word, count_has_label, count_total_num

word_amb = []
has_label = []
total_count = []

for i in ['今天', '昨天', '明天', '後天', '前天', '早上', '中午', '晚上', '凌晨']:
    a, b, c = find_out_aidea(i)
    word_amb.append(a)
    has_label.append(b)
    total_count.append(c)

df_final = pd.DataFrame({'word':word_amb, 'label_count':has_label, 'total_count':total_count})

In [42]:
def percent(row):
    return str(int(row.label_count/row.total_count*100))+'%'
df_final['百分比'] = df_final.apply(percent, 1)

In [43]:
df_final

,word,label_count,total_count,百分比
0,今天,171,450,38%
1,昨天,21,37,56%
2,明天,16,33,48%
3,後天,11,19,57%
4,前天,28,30,93%
5,早上,96,153,62%
6,中午,12,25,48%
7,晚上,27,105,25%
8,凌晨,1,1,100%
